In [ ]:
# Imports
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)

from src import *
import scipy.sparse.linalg as spsl
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Hamiltonian Parameters
num_qubits = 15
J = 1
h = 1

# Algorithm Parameters
max_energy_level = 3
kd_ratio = 2.5
noise_threshold = 1e-2
epsilon = 1e-3
K_values = list(range(0,500,5))
delta_t = 0.09
num_modmd_observables = 7
num_trials = 20

In [ ]:
# Generate Hamiltonian and get true eigenenergies
sparse_hamiltonian = heisenberg_hamiltonian(num_qubits,J,h).to_matrix(sparse=True)

v, w = spsl.eigsh(sparse_hamiltonian,k=20,which = 'SA')
true_eigenenergies = np.unique(np.round(v,8))[:max_energy_level+1]

In [ ]:
# Construct reference state
reference_state = bitstring_superposition_state(num_qubits, ['000000000000000','100000000000000','110000000000000','111000000000000'])
Z_tot = total_spin_operator(num_qubits).to_matrix(sparse=True)
reference_state_total_spin = np.real(reference_state@Z_tot@reference_state)

# Get evolved reference states
max_K = K_values[-1]
max_d = int(max_K/kd_ratio)
time_evolution_operator = -1j*sparse_hamiltonian*delta_t
evolved_reference_states = spsl.expm_multiply(time_evolution_operator,reference_state,start=0,stop=max_d+max_K+1,num = max_d+max_K+2)

In [ ]:
# # ODMD Results
odmd_observables = [SparsePauliOp('I' * num_qubits).to_matrix(sparse=True)]
odmd_results = []

X_elements = generate_X_elements(odmd_observables,max_d,max_K,reference_state,evolved_reference_states)

for trial in range(num_trials):
    
    gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
    noisy_X_elements = X_elements + gaussian_noise

    odmd_results.append(varying_K_results(len(odmd_observables),noise_threshold,noisy_X_elements,delta_t,K_values,kd_ratio,max_energy_level))

In [ ]:
# MODMD Results
modmd_results = []

for trial in range(num_trials):
    
    modmd_observables = odmd_observables + random_one_local_paulis(num_qubits,num_modmd_observables-1)

    X_elements = generate_X_elements(modmd_observables,max_d,max_K,reference_state,evolved_reference_states)
    gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
    noisy_X_elements = X_elements + gaussian_noise

    modmd_results.append(varying_K_results(len(modmd_observables),noise_threshold,noisy_X_elements,delta_t,K_values,kd_ratio,max_energy_level))

In [ ]:
# Compute errors
absolute_odmd_errors = np.array([np.abs(odmd_results[i] - true_eigenenergies) for i in range(num_trials)])
absolute_modmd_errors = np.array([np.abs(modmd_results[i] - true_eigenenergies) for i in range(num_trials)])

In [ ]:
# Plotting
algorithm = 'ODMD'
colors = get_color_set('Heisenberg')
labels = [r'$|\delta E_0|$',r'$|\delta E_1|$',r'$|\delta E_2|$',r'$|\delta E_3|$']
matplotlib.rcParams.update({'font.size': 14})

for energy_level in range(max_energy_level+1):

    if algorithm == 'ODMD':
        odmd_average = np.average(absolute_odmd_errors,0)[:,energy_level]
        odmd_std = np.std(absolute_odmd_errors,0)[:,energy_level]
        plt.semilogy(K_values, odmd_average,'--',color = colors[energy_level],label = labels[energy_level])
        plt.fill_between(K_values,odmd_average,odmd_average + odmd_std,color = colors[energy_level],alpha = .3)
        
    elif algorithm == 'MODMD':
        modmd_average = np.average(absolute_modmd_errors,0)[:,energy_level]
        modmd_std = np.std(absolute_modmd_errors,0)[:,energy_level]
        plt.semilogy(K_values,modmd_average,color = colors[energy_level],label = labels[energy_level])
        plt.fill_between(K_values,modmd_average,modmd_average + modmd_std,color = colors[energy_level],alpha = .3)

plt.axhline(epsilon, color = '#95ab9b', linestyle = '--', label = r'$\epsilon_\text{noise}$')
plt.xlabel(r'K ($\propto \text{Simulation Time})$')
plt.ylabel(r'Absolute Error')
plt.legend(fontsize=14, framealpha = 0)